Task is to import the most current violations from the NY State Restaurant Inspections.  The information below is from a direct connection to a CSV download into the IBM DSX environment as an Asset.  The code was changed to apply common variable names with successive coding.

In [1]:
# The code was removed by DSX for sharing.

,FACILITY,ADDRESS,LAST INSPECTED,VIOLATIONS,TOTAL # CRITICAL VIOLATIONS,TOTAL #CRIT. NOT CORRECTED,TOTAL # NONCRITICAL VIOLATIONS,DESCRIPTION,LOCAL HEALTH DEPARTMENT,COUNTY,...,PERMIT EXPIRATION DATE,PERMITTED (D/B/A),PERMITTED CORP. NAME,PERM. OPERATOR LAST NAME,PERM. OPERATOR FIRST NAME,NYS HEALTH OPERATION ID,INSPECTION TYPE,INSPECTION COMMENTS,FOOD SERVICE FACILITY STATE,Location1
0,CARVEL,"49 EAST ROUTE 59, NANUET",12/08/2016,Item 12C- Plumbing and sinks not properly si...,0.0,0.0,2.0,Food Service Establishment - Food Service Esta...,Rockland County,ROCKLAND,...,11/30/2018,NaN,YANG'S CARVEL INC.,YANG,JI,822685,Inspection,NaN,NY,"(41.096204, -74.009739)"
1,F L BOCES VOCATIONAL CENTER,"3501 COUNTY ROAD 20, STANLEY",12/15/2016,No violations found.,0.0,0.0,0.0,Institutional Food Service - School K-12 Food ...,Geneva District Office,ONTARIO,...,05/31/2018,NaN,WAYNE-FINGER LAKES BOCES,HENRY,KEITH,326577,Inspection,NaN,NY,"(42.86233, -77.111917)"
2,AMERICAN LEGION POST #1122,"840 MCLEAN AVENUE, YONKERS",12/16/2016,"Item 8A- Food not protected during storage,...",0.0,0.0,3.0,Food Service Establishment - Food Service Esta...,Westchester County,WESTCHESTER,...,05/31/2018,AMERICAN LEGION POST #1122,AMERICAN LEGION INC. POST #1122,AMERICAN LEGION INC. POST #112,,443323,Inspection,MAIL REPORT.,NY,"(40.905959, -73.870639)"
3,MASSAPEQUA SD - EAST LAKE ELEM SCH,"154 EAST LAKE AVENUE, MASSAPEQUA PARK",12/20/2016,No violations found.,0.0,0.0,0.0,Institutional Food Service - School K-12 Food ...,Nassau County,NASSAU,...,01/31/2019,NaN,MASSAPEQUA BOARD OF EDUCATION,HECKELMAN,PAUL,576962,Inspection,"Score ""A""\nFMTC Laura Feely exp 6/18",NY,"(40.699875, -73.436889)"
4,SID JACOBSON JCC ADULT DAY PROGRAM,"300 FOREST DRIVE, EAST HILLS",01/05/2017,No violations found.,0.0,0.0,0.0,Institutional Food Service - Adult Feeding (No...,Nassau County,NASSAU,...,02/28/2018,NaN,SID JACOBSON JCC,NaN,NaN,676666,Inspection,"1.40 meals, Rating ""A"", Food Managers Certific...",NY,"(40.805339, -73.637404)"


In [2]:
nyrcols = nyr[['FACILITY', 'TOTAL # CRITICAL VIOLATIONS','Location1']]
nyrcols.head()

,FACILITY,TOTAL # CRITICAL VIOLATIONS,Location1
0,CARVEL,0.0,"(41.096204, -74.009739)"
1,F L BOCES VOCATIONAL CENTER,0.0,"(42.86233, -77.111917)"
2,AMERICAN LEGION POST #1122,0.0,"(40.905959, -73.870639)"
3,MASSAPEQUA SD - EAST LAKE ELEM SCH,0.0,"(40.699875, -73.436889)"
4,SID JACOBSON JCC ADULT DAY PROGRAM,0.0,"(40.805339, -73.637404)"


The original example used the spark libriary.  As I couldn't get this to run, I installed the pyspark liabrairy with the code in the next line.  From there I imported teh SparkSession and SQL Context based on the documentation.  I used the generic SparkSession instance generation from the documentation and it worked.

In [4]:
# This code only needs to run once during a VM instance.  It is retained in the permanent 
# memory of the instance for use by the CLI - on initialization (new VM) the pip will run
# normaly by first downloading the packages and then installing them.  If the pyspark has
# already been installed in the instance the code will return two confirmations that the
# environment was not upgraded as it is not direclty reuqired.

!pip install --user pyspark

Requirement not upgraded as not directly required: pyspark in /home/dsxuser/.local/lib/python3.5/site-packages
Requirement not upgraded as not directly required: py4j==0.10.6 in /home/dsxuser/.local/lib/python3.5/site-packages (from pyspark)


In [5]:
# The instance of the variable 'spark' is quite a bit different than how it is done in 
# the spark librairy.  In the spark librairy the code imports spark and then sets the
# DF to 'spark.createDataFrame(var)' without the need to call '.getOrCrete()'.  I'll need
# to test the generatation of teh instance in greater detail.  The documentation describes
# that SparkSession adn SQLContext both initiate instances using 'spark' and 'sc'.  I was
# unable to get either of those to work.  I did note a discussion in StackOverflow noting
# that runing pyspark.sql in the DSX environment had a known error regarding the generation
# of 'sc', however, the OP also claimed that he got it to run without the adjustments I've
# made.

from pyspark.sql import SparkSession, SQLContext

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
nyrDF = spark.createDataFrame(nyrcols)
nyrDF.registerTempTable("nyrDF")

The '.registerTempTable' for teh 'nyrDF' dataframe creates a relational database and registers it for use with a subsequent SQL statement.  I tried coding the instance of teh SQLContext object using a much simplified expression ('spark.builder.getOrCreate()') and it worked.  It would seem that you could just as easily create the SparkSession in the same method.  The followign quote will convert the nyvDF into a SQL accessible/structured dataset. 

In [6]:
sqlContext = spark.builder.getOrCreate()

# at a later date, I'll need to understand how the sqlContext quary structure works.

query = """
select 
    FACILITY, 
    trim(regexp_extract(location1, '(\\\()(.*),(.*)(\\\))',2)) as lat, 
    trim(regexp_extract(location1, '(\\\()(.*),(.*)(\\\))',3)) as lon,
    cast(`TOTAL # CRITICAL VIOLATIONS` as int) as Violations
from nyrDF 
order by Violations desc
limit 1000
"""

nyvDF = sqlContext.sql(query)
nyvDF.show(10)

+--------------------+---------+----------+----------+
|            FACILITY|      lat|       lon|Violations|
+--------------------+---------+----------+----------+
|MOHAWK VALLEY COM...|43.075939|-75.219801|        14|
|ELIZAVILLE DINER ...|42.047504|-73.803177|        12|
|PEPPINOS KITCHEN ...|42.528962|-73.679651|        11|
|LOBSTER POT RESTA...|43.421017| -73.71429|        10|
|APOLLO RESTAURANT...|42.442005|-76.485183|        10|
|LA FIESTA MEXICAN...| 42.86481|-73.778805|        10|
|MALTA DINER      ...|42.969989|-73.792906|        10|
|CITY TAVERN      ...|43.021289|-77.440138|         9|
|HALFMOON DINER   ...|42.837692|-73.743282|         9|
|Camp Chateaugay &...|44.767719|-73.982924|         9|
+--------------------+---------+----------+----------+
only showing top 10 rows



In [7]:
import brunel
nyvPan = nyvDF.toPandas()
%brunel map ('NY') + data('nyvPan') x(lon) y(lat) color(Violations) tooltip(FACILITY)

<IPython.core.display.Javascript object>